In [87]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from google.colab import drive
import sys, os, shutil
from pathlib import Path
drive.mount("/content/drive", force_remount=True); my_drive = Path('/content/drive/My Drive/');
colabase = my_drive/'Colab Notebooks/'; course = my_drive/'github/fastai_course-v3/'
dlff = my_drive/'deeplearning-ff/'; pkl_path= dlff/'pickles'
os.chdir(dlff) #sys.path.append(str(dlff)) # cd {dlff}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Mounted at /content/drive


## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [0]:
#export
from exports.lg_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [0]:
x_train,y_train,x_valid,y_valid = get_data()

In [90]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [0]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [92]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [0]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [0]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [95]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [0]:
# num hidden
nh = 50

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

In [0]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [0]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [99]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [0]:
#export
def lin_man(x, w, b): 
  """Manual linear transform"""
  return x@w + b 

In [0]:
t = lin_man(x_valid, w1, b1)
# with this, each elt of matrix is a sum of m elts 
# for the sake of arg, if u say indep stdnorm, var = m * var_i so stdev = sqrt(m) * stdev_i
# so we have to divide by sqrt(m)

In [102]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(-0.1839), tensor(0.9872))

In [0]:
def relu_man(x): 
  """Manual ReLU function """
  return x.clamp_min(0.)

In [0]:
t = relu_man(lin_man(x_valid, w1, b1))

In [105]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3007), tensor(0.5135))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [0]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [0]:
# export
def kaiming_init(m,nh): 
  """Scratch Kaiming He init. Use as w1 = kaiming_init(m,nh)"""
  return torch.randn(m,nh)*math.sqrt(2./m)

In [108]:
w1 = kaiming_init(m,nh)
w1.mean(),w1.std()

(tensor(0.0006), tensor(0.0507))

In [109]:
t = relu_man(lin_man(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5791), tensor(0.8557))

In [0]:
#export
from torch.nn import init

In [0]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') #normally fan_in to keep unit var of fwd pass. see below
t = relu_man(lin_man(x_valid, w1, b1))

In [0]:
init.kaiming_normal_??

In [113]:
w1.mean(),w1.std()

(tensor(-7.0117e-06), tensor(0.0504))

In [114]:
t.mean(),t.std()

(tensor(0.5542), tensor(0.8103))

In [115]:
w1.shape

torch.Size([784, 50])

In [0]:
import torch.nn

In [117]:
torch.nn.Linear(m,nh).weight.shape # there is a transpose inside the pytorch Linear output = input.matmul(weight.t())
# thats why we use fan_out io fan_in ( we are 'tricking' the funcs to get unit var fwd pass)

torch.Size([50, 784])

In [0]:
torch.nn.Linear.forward??

In [0]:
torch.nn.functional.linear??
# F.linear means this F = torch.nn.functional; torch.nn.Linear calls F.linear

In [0]:
torch.nn.Conv2d??

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??
# whats with the sqrt(5)?

In [0]:
#export
def relu_tweak(x): 
  """Tweaked ReLU: what if we substract 0.5 to get relu mean to 0"""
  return x.clamp_min(0.) - 0.5

In [123]:
# kaiming init / he init for relu
# w1 = torch.randn(m,nh)*math.sqrt(2./m )
w1 = kaiming_init(m,nh)
t1 = relu_tweak(lin_man(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(-0.0272), tensor(0.7588))

In [0]:
def model(xb):
    l1 = lin_man(xb, w1, b1)
    l2 = relu_tweak(l1)
    l3 = lin_man(l2, w2, b2)
    return l3

In [125]:
%timeit -n 10 _=model(x_valid)

10 loops, best of 3: 19.6 ms per loop


In [0]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [127]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [0]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [0]:
y_train,y_valid = y_train.float(),y_valid.float()

In [0]:
preds = model(x_train)

In [131]:
preds.shape

torch.Size([50000, 1])

In [132]:
mse(preds, y_train)

tensor(28.7992)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

In [0]:
#export
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer; 
    # inp here is 'out' of relu; so we store the grad there to pass it through, and use chain rule
    # this is dMSE / dyhat 
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]
    # 2/N (yhat-y); yhat is inp, inp/shape is N (numb classes/dim output) 

In [0]:
#export
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    # this is dRelu(z)/dz * dMSE/dRelu: accumulating chain rule
    inp.g = (inp>0).float() * out.g 
    # out.g contains inp.g from mse_grad stored by mse_grad
    # inp.g is stored for lin_grad

In [0]:
#export
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input,weight and biase
    inp.g = out.g @ w.t()     # grad of matrix prod = matri prod with transpose
    # Creating a giant outer product, just to sum it, is inefficient!
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0) #sum over training examples (1/m baked in from mse_grad)
    b.g = out.g.sum(0) #sum over training examples (1/m baked in from mse_grad)

In [0]:
#export
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu_tweak(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2) # out.g contains grad of next layer
    relu_grad(l1, l2)         # l2.g contains grad of next layer
    lin_grad(inp, l1, w1, b1) # l1.g contains grad of next layer

In [0]:
forward_and_backward(x_train, y_train)

In [0]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [0]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [0]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu_tweak(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [0]:
loss = forward(xt2, y_train)

In [0]:
loss.backward()

In [0]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [0]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [0]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [0]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [0]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [149]:
%time loss = model(x_train, y_train)

CPU times: user 114 ms, sys: 0 ns, total: 114 ms
Wall time: 116 ms


In [150]:
%time model.backward()

CPU times: user 4.58 s, sys: 48 ms, total: 4.63 s
Wall time: 4.64 s


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [0]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [0]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [0]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [0]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [158]:
%time loss = model(x_train, y_train)

CPU times: user 113 ms, sys: 744 µs, total: 113 ms
Wall time: 113 ms


In [159]:
%time model.backward()

CPU times: user 229 ms, sys: 1.9 ms, total: 231 ms
Wall time: 231 ms


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g #includes potentially summing over batch. 
        # @ and matmult handle tensor of rank over 2.
        self.b.g = out.g.sum(0)

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [163]:
%time loss = model(x_train, y_train)

CPU times: user 113 ms, sys: 681 µs, total: 114 ms
Wall time: 113 ms


In [164]:
%time model.backward()

CPU times: user 233 ms, sys: 1.93 ms, total: 235 ms
Wall time: 235 ms


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [0]:
#export
from torch import nn

In [0]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [0]:
model = Model(m, nh, 1)

In [169]:
%time loss = model(x_train, y_train)

CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 103 ms


In [170]:
%time loss.backward()

CPU times: user 79.6 ms, sys: 2.8 ms, total: 82.4 ms
Wall time: 81.7 ms


## Export

In [171]:
!pip install fire

In [172]:
! python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exports/lg_02.py
